In [39]:
import pyvista as pv
import meshio
import numpy as np
import psycopg2
import hdf5storage
import h5py
import pickle
import logging
from sys import getsizeof
from dotenv import dotenv_values

pv.global_theme.transparent_background = True


config = dotenv_values("../.env")
# Filepaths for script
initMeshPath = config["currentDirectory"] +"meshes/fullAssemblyContiguous/fullFishContiguous.msh"
tempMeshPath = config["backupDirectory"] + "data/visualizations/holdpls/"
outputPath = config["currentDirectory"] + "data/visualizations/controlExperiments/"


In [40]:
# Helper functions for working with database
def get_db_connection():
    conn = psycopg2.connect(
        dbname='simDB',
        user='user',
        password='password',
        host='localhost',
        port='5432'
    )
    return conn

def fetch_trial_data_singleTimestep(conn, trial_id, n=1, start_timestep=0, num_timesteps=500):
    try:
        cur = conn.cursor()
        
        # Fetch data for the given trial_id starting from start_timestep
        query = '''
        SELECT timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat
        FROM simulation_data
        WHERE trial_id = %s AND timestep >= %s AND timestep < %s
        ORDER BY timestep ASC;
        '''
        cur.execute(query, (trial_id, start_timestep, start_timestep + 1))
        rows = cur.fetchall()
        
        # Deserialize data
        data = []
        for row in rows:
            timestep, simulation_time, input_data_bin, output_data_bin, state_data_bin, y_ref_bin, x_hat_bin = row
            input_data = np.array(pickle.loads(input_data_bin))
            output_data = np.array(pickle.loads(output_data_bin))
            state_data = np.array(pickle.loads(state_data_bin))
            y_ref = np.array(pickle.loads(y_ref_bin))
            x_hat = np.array(pickle.loads(x_hat_bin))
            data.append((timestep, simulation_time, input_data, output_data,state_data, y_ref, x_hat))
        
        cur.close()
        return data
    except Exception as e:
        print(f"Error fetching data for trial_id {trial_id}: {e}")
        return None
    


In [41]:
# Trial Parameters
trial_ids = [162, 163]
timesteps = 1500


In [42]:
### Get offsets for reference trajectory
# Load data from database
conn = get_db_connection()
data = fetch_trial_data_singleTimestep(conn, trial_ids[0], 1, start_timestep=0, num_timesteps=1)
conn.close()
timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
# Get the reference trajectory
refOffsets = np.copy(output_data)


In [43]:
# ###### Generate mp4 of fish swimming in isometric view along with reference trajectory for specified trials
# for trial_id in trial_ids:
#     # create plotter object
#     plotter = pv.Plotter()
#     # Open a mp4
#     plotter.open_movie(outputPath + f"Trial{trial_id}_isoView.mp4", framerate = 100, quality=7)
#     # Iterate through timesteps and add frames
#     for i in range(timesteps):
#         print(f"Trial {trial_id}, Timestep {i}")
#         ### Make surface from desired reference trajectory
#         # Load data from database
#         conn = get_db_connection()
#         data = fetch_trial_data_singleTimestep(conn, trial_id, 1, start_timestep=i, num_timesteps=1)
#         conn.close()
#         timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
#         # Check shape of reference trajectory
#         uncenteredRefTraj=y_ref + refOffsets
#         # Turn reference trajectory from 40x1 to 20x2 array by taking every other point 
#         x_refs = uncenteredRefTraj[::2]
#         z_refs = uncenteredRefTraj[1::2]
#         referenceCurve = np.column_stack((x_refs, np.zeros(np.size(x_refs)), z_refs))
#         # print(np.shape(referenceCurve))
#         path = referenceCurve
#         offsetPath = np.copy(path)
#         offsetPath[:, 1] = offsetPath[:, 1] - 102
#         #### Draw Surface for reference trajectory
#         # Number of samples (in depth dir) and number of points along centerline 
#         nsamples = 3
#         ntraces = 20  
#         # Define the Z spacing of your 2D section
#         z_spacing = 50
#         # Create structured points draping down from the path
#         points = np.repeat(path, nsamples, axis=0)
#         # repeat the Z locations across
#         tp = np.arange(0, z_spacing * nsamples, z_spacing)
#         # Offset the Z locations 
#         tp = path[:, 1][:, None] - tp
#         points[:, 1] = tp.ravel()
#         #Make a StructuredGrid from the structured points
#         grid = pv.StructuredGrid()
#         grid.points = points
#         grid.dimensions = nsamples, ntraces, 1
#         # Add the data array - note the ordering
#         grid["values"] = np.ones((nsamples, ntraces), dtype=np.float64).ravel(order="F")

#         ### Plot actual centerline from output_data
#         # Load data from database
#         conn = get_db_connection()
#         data = fetch_trial_data_singleTimestep(conn, trial_id, 1, start_timestep=i, num_timesteps=1)
#         conn.close()
#         timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
#         # Check shape of output data
#         uncenteredOutputTraj=output_data
#         # Turn output trajectory from 40x1 to 20x2 array by taking every other point
#         x_outs = uncenteredOutputTraj[::2]
#         z_outs = uncenteredOutputTraj[1::2]
#         outputCurve = np.column_stack((x_outs, np.zeros(np.size(x_outs)), z_outs))
#         # print(np.shape(outputCurve))
#         offsetOutputCurve = np.copy(outputCurve)
#         offsetOutputCurve[:, 1] = offsetOutputCurve[:, 1] - 102


        

#         # load mesh from .vtu file in tempMeshPath
#         mesh = pv.read(tempMeshPath + f"Trial{trial_id}_step{i}.vtu")
#         # add fish mesh to plotter
#         plotter.add_mesh(mesh, show_scalar_bar=False, name = 'mesh') # add the fish body
#         plotter.add_mesh(grid, clim=[-1, 1],show_scalar_bar=False, name = 'reference', opacity=0.5)
#         # add reference trajectory to plotter with thickness of 5 and large markers
#         opacities = np.linspace(1, 0.05, len(offsetPath))
#         plotter.add_mesh(pv.PolyData(offsetPath), color='orange', line_width=5, name = 'refPath', opacity=opacities, render_points_as_spheres=True, point_size=10)
#         # add output trajectory to plotter with thickness of 5 and large markers
#         plotter.add_mesh(pv.PolyData(offsetOutputCurve), color='blue', line_width=5, name = 'outputPath', opacity=opacities, render_points_as_spheres=True, point_size=10)
#         # Set the camera position - Isometric view
#         plotter.camera_position = 'yz'
#         plotter.camera.roll = 180
#         plotter.camera.azimuth = 40#90 #
#         plotter.camera.elevation = 50#0 #50
#         plotter.enable_parallel_projection()
#         # # Set the camera position - Top View Down
#         # plotter.camera_position = 'yz'
#         # plotter.camera.roll = 180
#         # plotter.camera.azimuth = 90 #
#         # plotter.camera.elevation = 90 #50
#         # plotter.enable_parallel_projection()


#         # write frame to gif
#         plotter.write_frame()
#     # Close the gif
#     plotter.close()

    



In [44]:
import matplotlib.pyplot as plt

opacities = np.linspace(1, 0.2, 20)
###### Generate mp4 of fish swimming with top and isometric views along with reference trajectory for specified trials
for trial_id in trial_ids:
    # create plotter objects
    plotter = pv.Plotter(shape=(2,1), border=False, row_weights = [0.4,1], window_size=[1000, 1000])
    # Open a mp4
    plotter.open_movie(outputPath + f"Trial{trial_id}_combinedView.mp4", framerate=100, quality=7)
    
    # Iterate through timesteps and add frames
    for i in range(timesteps):
        print(f"Trial {trial_id}, Timestep {i}")
        
        ### Make surface from desired reference trajectory
        # Load data from database
        conn = get_db_connection()
        data = fetch_trial_data_singleTimestep(conn, trial_id, 1, start_timestep=i, num_timesteps=1)
        conn.close()
        timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
        
        # Check shape of reference trajectory
        uncenteredRefTraj = y_ref + refOffsets
        # Turn reference trajectory from 40x1 to 20x2 array by taking every other point 
        x_refs = uncenteredRefTraj[::2]
        z_refs = uncenteredRefTraj[1::2]
        referenceCurve = np.column_stack((x_refs, np.zeros(np.size(x_refs)), z_refs))
        path = referenceCurve
        offsetPath = np.copy(path)
        offsetPath[:, 1] = offsetPath[:, 1] - 122
        
        #### Draw Surface for reference trajectory
        # Number of samples (in depth dir) and number of points along centerline 
        nsamples = 3
        ntraces = 20  
        # Define the Z spacing of your 2D section
        z_spacing = 60
        # Create structured points draping down from the path
        points = np.repeat(path, nsamples, axis=0)
        # repeat the Z locations across
        tp = np.arange(0, z_spacing * nsamples, z_spacing)
        # Offset the Z locations 
        tp = path[:, 1][:, None] - tp
        points[:, 1] = tp.ravel()
        # Make a StructuredGrid from the structured points
        grid = pv.StructuredGrid()
        grid.points = points
        grid.dimensions = nsamples, ntraces, 1
        # Add the data array - note the ordering
        grid["values"] = np.ones((nsamples, ntraces), dtype=np.float64).ravel(order="F")
        
        ### Plot actual centerline from output_data
        # Load data from database
        conn = get_db_connection()
        data = fetch_trial_data_singleTimestep(conn, trial_id, 1, start_timestep=i, num_timesteps=1)
        conn.close()
        timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
        
        # Check shape of output data
        uncenteredOutputTraj = output_data
        # Turn output trajectory from 40x1 to 20x2 array by taking every other point
        x_outs = uncenteredOutputTraj[::2]
        z_outs = uncenteredOutputTraj[1::2]
        outputCurve = np.column_stack((x_outs, np.zeros(np.size(x_outs)), z_outs))
        offsetOutputCurve = np.copy(outputCurve)
        offsetOutputCurve[:, 1] = offsetOutputCurve[:, 1] - 122
        
        # load mesh from .vtu file in tempMeshPath
        mesh = pv.read(tempMeshPath + f"Trial{trial_id}_step{i}.vtu")
        
        # Add meshes to top view window
        plotter.subplot(0, 0)
        plotter.add_mesh(mesh, show_scalar_bar=False, name='mesh')
        plotter.add_mesh(grid, clim=[-1, 1], show_scalar_bar=False, name='reference', opacity=0.5)
        plotter.add_mesh(pv.PolyData(offsetPath), color='orange', line_width=5, name='refPath', opacity=opacities, render_points_as_spheres=True, point_size=10)
        plotter.add_mesh(pv.PolyData(offsetOutputCurve), color='blue', line_width=5, name='outputPath', opacity=opacities, render_points_as_spheres=True, point_size=10)
        plotter.camera_position = 'yz'
        plotter.camera.roll = 180
        plotter.camera.azimuth = 90
        plotter.camera.elevation = 90
        plotter.enable_parallel_projection()
        # change bounds to be 0, 0, -200, 200, -200, 200
        plotter.camera_position = [
            (-558.609577998519, -2226.5518997921977, -13.31930130655202),
            (-558.609577998519, -19.333351135253906, -13.31930130655202),
            (0.0, -1.0, -2.220446049250313e-16)]
        plotter.camera.zoom(3)

        # Add meshes to isometric view window
        plotter.subplot(1, 0)
        plotter.add_mesh(mesh, show_scalar_bar=False, name='mesh')
        plotter.add_mesh(grid, clim=[-1, 1], show_scalar_bar=False, name='reference', opacity=0.5)
        plotter.add_mesh(pv.PolyData(offsetPath), color='orange', line_width=5, name='refPath', opacity=opacities, render_points_as_spheres=True, point_size=10)
        plotter.add_mesh(pv.PolyData(offsetOutputCurve), color='blue', line_width=5, name='outputPath', opacity=opacities, render_points_as_spheres=True, point_size=10)
        plotter.camera_position = 'yz'
        plotter.camera.roll = 180
        plotter.camera.azimuth = 40
        plotter.camera.elevation = 50
        plotter.enable_parallel_projection()
        plotter.camera_position = [
            (528.2333916563371, -1710.160855083042, 898.6502336784569),
            (-558.609577998519, -19.333351135253906, -13.31930130655202),
            (0.0, -1.0, -2.220446049250313e-16)]
        plotter.camera.zoom(1.25)        


        # plotter.show_bounds(all_edges=False)        
        # write frame to gif
        plotter.write_frame()
    # Close the gif
    plotter.close()


Trial 162, Timestep 0


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1000, 1000) to (1008, 1008) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Trial 162, Timestep 1
Trial 162, Timestep 2
Trial 162, Timestep 3
Trial 162, Timestep 4
Trial 162, Timestep 5
Trial 162, Timestep 6
Trial 162, Timestep 7
Trial 162, Timestep 8
Trial 162, Timestep 9
Trial 162, Timestep 10
Trial 162, Timestep 11
Trial 162, Timestep 12
Trial 162, Timestep 13
Trial 162, Timestep 14
Trial 162, Timestep 15
Trial 162, Timestep 16
Trial 162, Timestep 17
Trial 162, Timestep 18
Trial 162, Timestep 19
Trial 162, Timestep 20
Trial 162, Timestep 21
Trial 162, Timestep 22
Trial 162, Timestep 23
Trial 162, Timestep 24
Trial 162, Timestep 25
Trial 162, Timestep 26
Trial 162, Timestep 27
Trial 162, Timestep 28
Trial 162, Timestep 29
Trial 162, Timestep 30
Trial 162, Timestep 31
Trial 162, Timestep 32
Trial 162, Timestep 33
Trial 162, Timestep 34
Trial 162, Timestep 35
Trial 162, Timestep 36
Trial 162, Timestep 37
Trial 162, Timestep 38
Trial 162, Timestep 39
Trial 162, Timestep 40
Trial 162, Timestep 41
Trial 162, Timestep 42
Trial 162, Timestep 43
Trial 162, Timestep 

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1000, 1000) to (1008, 1008) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Trial 163, Timestep 1
Trial 163, Timestep 2
Trial 163, Timestep 3
Trial 163, Timestep 4
Trial 163, Timestep 5
Trial 163, Timestep 6
Trial 163, Timestep 7
Trial 163, Timestep 8
Trial 163, Timestep 9
Trial 163, Timestep 10
Trial 163, Timestep 11
Trial 163, Timestep 12
Trial 163, Timestep 13
Trial 163, Timestep 14
Trial 163, Timestep 15
Trial 163, Timestep 16
Trial 163, Timestep 17
Trial 163, Timestep 18
Trial 163, Timestep 19
Trial 163, Timestep 20
Trial 163, Timestep 21
Trial 163, Timestep 22
Trial 163, Timestep 23
Trial 163, Timestep 24
Trial 163, Timestep 25
Trial 163, Timestep 26
Trial 163, Timestep 27
Trial 163, Timestep 28
Trial 163, Timestep 29
Trial 163, Timestep 30
Trial 163, Timestep 31
Trial 163, Timestep 32
Trial 163, Timestep 33
Trial 163, Timestep 34
Trial 163, Timestep 35
Trial 163, Timestep 36
Trial 163, Timestep 37
Trial 163, Timestep 38
Trial 163, Timestep 39
Trial 163, Timestep 40
Trial 163, Timestep 41
Trial 163, Timestep 42
Trial 163, Timestep 43
Trial 163, Timestep 

In [45]:
# ###### Generate mp4 of fish swimming in Top down view along with reference trajectory for specified trials
# for trial_id in trial_ids:
#     # create plotter object
#     plotter = pv.Plotter()
#     # Open a mp4
#     plotter.open_movie(outputPath + f"Trial{trial_id}_topView.mp4", framerate = 100, quality=7)
#     # Iterate through timesteps and add frames
#     for i in range(timesteps):
#         print(f"Trial {trial_id}, Timestep {i}")
#         ### Make surface from desired reference trajectory
#         # Load data from database
#         conn = get_db_connection()
#         data = fetch_trial_data_singleTimestep(conn, trial_id, 1, start_timestep=i, num_timesteps=1)
#         conn.close()
#         timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
#         # Check shape of reference trajectory
#         uncenteredRefTraj=y_ref + refOffsets
#         # Turn reference trajectory from 40x1 to 20x2 array by taking every other point 
#         x_refs = uncenteredRefTraj[::2]
#         z_refs = uncenteredRefTraj[1::2]
#         referenceCurve = np.column_stack((x_refs, np.zeros(np.size(x_refs)), z_refs))
#         # print(np.shape(referenceCurve))
#         path = referenceCurve
#         offsetPath = np.copy(path)
#         offsetPath[:, 1] = offsetPath[:, 1] - 200
#         #### Draw Surface for reference trajectory
#         # Number of samples (in depth dir) and number of points along centerline 
#         nsamples = 5
#         ntraces = 20  

#         # Define the Z spacing of your 2D section
#         z_spacing = 50
#         # Create structured points draping down from the path
#         points = np.repeat(path, nsamples, axis=0)
#         # repeat the Z locations across
#         tp = np.arange(0, z_spacing * nsamples, z_spacing)
#         # Offset the Z locations 
#         tp = path[:, 1][:, None] - tp
#         points[:, 1] = tp.ravel()
#         #Make a StructuredGrid from the structured points
#         grid = pv.StructuredGrid()
#         grid.points = points
#         grid.dimensions = nsamples, ntraces, 1

#         # Add the data array - note the ordering
#         grid["values"] = np.ones((nsamples, ntraces), dtype=np.float64).ravel(order="F")

#         # load mesh from .vtu file in tempMeshPath
#         mesh = pv.read(tempMeshPath + f"Trial{trial_id}_step{i}.vtu")
#         # add fish mesh to plotter
#         plotter.add_mesh(mesh, show_scalar_bar=False, name = 'mesh') # add the fish body
#         plotter.add_mesh(grid, clim=[-1, 1],show_scalar_bar=False, name = 'reference', opacity=0.5)
#         # add reference trajectory to plotter with thickness of 5 and large markers
#         plotter.add_mesh(pv.PolyData(offsetPath), color='orange', line_width=5, name = 'refPath', opacity=1, render_points_as_spheres=True, point_size=10)
#         # Set the camera position - Top View Down
#         plotter.camera_position = 'yz'
#         plotter.camera.roll = 180
#         plotter.camera.azimuth = 90 #
#         plotter.camera.elevation = 90 #50
#         plotter.enable_parallel_projection()

#         # write frame to gif
#         plotter.write_frame()
#     # Close the gif
#     plotter.close()

    

